In [1]:
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation
from datetime import datetime
import os
import importlib
import qlbmlib

In [7]:
def get_gaussian_initial_distribution(sites: int, 
                                   center: float,
                                   width: float,
                                   amplitude: float = 0.9,
                                   background: float = 0.1) -> NDArray[np.float64]:
    """
    Create a Gaussian hill initial distribution.
    
    Args:
        sites: Number of lattice sites
        center: Center position of the Gaussian (in lattice units)
        width: Width (standard deviation) of the Gaussian
        amplitude: Peak height of the Gaussian
        background: Background density level
        
    Returns:
        density: Initial density distribution with a Gaussian hill
    """
    x = np.arange(sites)
    density = background + amplitude * np.exp(-((x - center) ** 2) / (2 * width ** 2))
    return density

def get_uniform_velocity_field(sites: int, velocity: float = 0.2) -> NDArray[np.float64]:
    """
    Create a uniform velocity field.
    
    Args:
        sites: Number of lattice sites
        velocity: Uniform velocity magnitude
        
    Returns:
        velocity_field: Array of shape (1, sites) containing the velocity
    """
    return np.full((1, sites), velocity)

In [8]:
def save_simulation_snapshots_1d(classical_file: str, quantum_file: str, 
                               iterations: list[int], output_path: str) -> None:
    """Save simulation snapshots comparing classical and quantum results.
    
    Args:
        classical_file: Path to classical simulation CSV file
        quantum_file: Path to quantum simulation CSV file
        iterations: List of iterations to display
        output_path: Path to save the figure
    """
    df_classical = pd.read_csv(classical_file, header=None)
    df_quantum = pd.read_csv(quantum_file, header=None)
    
    if max(iterations) >= len(df_classical):
        raise ValueError(f"Max iteration {max(iterations)} exceeds data length")
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Classical vs Quantum Simulation Comparison', fontsize=16)
    
    # Get global min and max for consistent y-axis
    vmin = min(df_classical.values.min(), df_quantum.values.min())
    vmax = max(df_classical.values.max(), df_quantum.values.max())
    
    for idx, iter_num in enumerate(iterations):
        ax = axes[idx // 2, idx % 2]
        x = np.arange(len(df_classical.iloc[0]))
        
        # Plot both classical and quantum results
        ax.plot(x, df_classical.iloc[iter_num], 'b-', label='Classical', alpha=0.7)
        ax.plot(x, df_quantum.iloc[iter_num], 'r--', label='Quantum', alpha=0.7)
        
        ax.set_title(f't = {iter_num}', pad=10)
        ax.set_xlabel('Position')
        ax.set_ylabel('Density')
        ax.set_ylim(vmin, vmax)
        ax.grid(True)
        if idx == 0:  # Only show legend on first subplot
            ax.legend()
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Snapshots saved to {output_path}")

def animate_density_evolution_1d(classical_file: str, quantum_file: str, 
                               interval: int = 100, repeat: bool = True) -> FuncAnimation:
    """Create an animation comparing classical and quantum evolution.
    
    Args:
        classical_file: Path to classical simulation CSV file
        quantum_file: Path to quantum simulation CSV file
        interval: Time between frames in milliseconds
        repeat: Whether to loop the animation
    """
    df_classical = pd.read_csv(classical_file, header=None)
    df_quantum = pd.read_csv(quantum_file, header=None)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(df_classical.iloc[0]))
    
    line_classical, = ax.plot([], [], 'b-', label='Classical', alpha=0.7)
    line_quantum, = ax.plot([], [], 'r--', label='Quantum', alpha=0.7)
    
    ax.set_title('Density Evolution')
    ax.set_xlabel('Position')
    ax.set_ylabel('Density')
    ax.grid(True)
    ax.legend()
    
    # Set fixed y-axis limits
    vmin = min(df_classical.values.min(), df_quantum.values.min())
    vmax = max(df_classical.values.max(), df_quantum.values.max())
    ax.set_xlim(0, len(x)-1)
    ax.set_ylim(vmin, vmax)
    
    def init():
        line_classical.set_data([], [])
        line_quantum.set_data([], [])
        return line_classical, line_quantum
    
    def update(frame):
        line_classical.set_data(x, df_classical.iloc[frame])
        line_quantum.set_data(x, df_quantum.iloc[frame])
        return line_classical, line_quantum
    
    anim = FuncAnimation(fig, update, frames=len(df_classical),
                        init_func=init, blit=True,
                        interval=interval, repeat=repeat)
    
    return anim

In [11]:
# Create 1D configuration
num_sites = 32
sites_1d = (num_sites,)

# D1Q3 lattice configuration
links = [[0], [-1], [1]]  # rest, left, right
weights = [2/3, 1/6, 1/6]  # Weights sum to 1
speed_of_sound = 1/np.sqrt(3)

# Create Gaussian hill initial distribution
initial_dist = get_gaussian_initial_distribution(
    sites=num_sites,
    center=num_sites/4,  # Start at 1/4 of the domain
    width=2.0,  # Width of the Gaussian
    amplitude=0.9,
    background=0.1
)

# Create sinusoidal velocity field
velocity_field = get_uniform_velocity_field(num_sites, velocity=0.3)

# Configuration for simulation (run for 20 iterations)
config_1d = [(50, velocity_field, links, weights, speed_of_sound)]

# Create experiment directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dirname = f"experiments/1DQ3Gaussian_{timestamp}"
os.makedirs(dirname, exist_ok=True)

# Define output files
classical_csv = f"{dirname}/classical.csv"
quantum_csv = f"{dirname}/quantum.csv"
comparison_fig = f"{dirname}/comparison.png"
rmse_plot = f"{dirname}/rmse_plot.png"

# Run both simulations
_ = qlbmlib.simulate_flow_classical(initial_dist, config_1d, classical_csv)
_ = qlbmlib.simulate_flow(initial_dist, config_1d, quantum_csv, True)

Classical simulation: iterations 0-50/50
Classical Iteration 1/50
Classical Iteration 2/50
Classical Iteration 3/50
Classical Iteration 4/50
Classical Iteration 5/50
Classical Iteration 6/50
Classical Iteration 7/50
Classical Iteration 8/50
Classical Iteration 9/50
Classical Iteration 10/50
Classical Iteration 11/50
Classical Iteration 12/50
Classical Iteration 13/50
Classical Iteration 14/50
Classical Iteration 15/50
Classical Iteration 16/50
Classical Iteration 17/50
Classical Iteration 18/50
Classical Iteration 19/50
Classical Iteration 20/50
Classical Iteration 21/50
Classical Iteration 22/50
Classical Iteration 23/50
Classical Iteration 24/50
Classical Iteration 25/50
Classical Iteration 26/50
Classical Iteration 27/50
Classical Iteration 28/50
Classical Iteration 29/50
Classical Iteration 30/50
Classical Iteration 31/50
Classical Iteration 32/50
Classical Iteration 33/50
Classical Iteration 34/50
Classical Iteration 35/50
Classical Iteration 36/50
Classical Iteration 37/50
Classi

In [14]:
importlib.reload(qlbmlib)

<module 'qlbmlib' from 'd:\\Data\\Codes\\Quantum\\Research\\qlbmlib.py'>

In [15]:
# Create visualization of snapshots
save_simulation_snapshots_1d(classical_csv, quantum_csv, [0, 15, 30, 50], comparison_fig)

# Create RMSE comparison plot
qlbmlib.save_rmse_comparison(classical_csv, quantum_csv, sites_1d, rmse_plot)

Snapshots saved to experiments/1DQ3Gaussian_20250607_135740/comparison.png
RMSE comparison saved to experiments/1DQ3Gaussian_20250607_135740/rmse_plot.png
RMSE comparison saved to experiments/1DQ3Gaussian_20250607_135740/rmse_plot.png
